In [1]:
from __future__ import print_function, division
import sys
from bs4 import BeautifulSoup
import requests
import unidecode

requests.__path__

import pandas as pd
import pickle

In [79]:
def scrape_page(item):
    url = "https://www.transfermarkt.com" + item.replace('profil', 'verletzungen')
    
    user_agent = {'User-agent': 'Mozilla/5.0'}
    response  = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    Dates = soup.find_all("td", {"class": "zentriert"})
    Injuries = soup.find_all("td", {"class": "hauptlink"})
    Lengths = soup.find_all("td", {"class": "rechts"})
    
    
    injuries = []
    lengths = []
    games_missed = []
    season = []
    date = []
    
    for i in range(len(Injuries)):
        if i % 2 == 0:
            injuries.append(Injuries[i].text)
        else:
            games_missed.append(Injuries[i].text)

    for i in range(len(Lengths)):
        if i % 2 == 0:
            lengths.append(Lengths[i].text)
    
    for i in range(len(Dates)):
        if i % 3 == 0:
            season.append(Dates[i].text)
        elif i % 3 == 1:
            date.append(Dates[i].text)

    return(list(zip(*[season, injuries, date, lengths, games_missed])))
    
    

In [77]:
players = pd.read_pickle('Transfermarkt_Historicals.pkl')

In [80]:
d = {}

In [85]:
not_appended = []

for i in (players['Profile_Link']):
    count = 0
    if count % 25 == 0:
        f = open("players_dictionary.pkl","wb")
        pickle.dump(d,f)
        f.close()
    try:
        d[i] = scrape_page(i)
        count += 1
    except IndexError:
        print(i)
        not_appended.append(i)
        continue

In [87]:
d

{'/kylian-mbappe/profil/spieler/342229': [('18/19',
   'Shoulder Injury',
   'Nov 21, 2018',
   '6 days',
   '1'),
  ('17/18', 'Ankle problems', 'Mar 1, 2018', '4 days', '1'),
  ('17/18', 'Head Injury', 'Jan 22, 2018', '7 days', '2'),
  ('17/18', 'Knee Problems', 'Aug 7, 2017', '3 days', '-'),
  ('16/17', 'Head Injury', 'Aug 11, 2016', '35 days', '7'),
  ('15/16', 'Testicular disruption', 'Jan 25, 2016', '21 days', '-')],
 '/neymar/profil/spieler/68290': [('18/19',
   'Torn Ligament',
   'Jun 7, 2019',
   '1 days',
   '-'),
  ('18/19', 'Foot Injury', 'Jan 24, 2019', '85 days', '18'),
  ('18/19', 'Rest ', 'Jan 11, 2019', '2 days', '1'),
  ('18/19', 'Adductor problems', 'Dec 3, 2018', '19 days', '4'),
  ('18/19', 'Adductor problems', 'Nov 21, 2018', '6 days', '1'),
  ('18/19', 'Rest ', 'Oct 19, 2018', '2 days', '1'),
  ('18/19', 'Rest ', 'Sep 13, 2018', '3 days', '1'),
  ('17/18', 'Metatarsal Fracture', 'Feb 26, 2018', '90 days', '16'),
  ('17/18', 'Thigh Problems', 'Jan 19, 2018', '6 da

In [88]:
f = open("injuries_dictionary.pkl","wb")
pickle.dump(d,f)
f.close()

In [89]:
players['Injury_History'] = players['Profile_Link'].map(d)

In [ ]:
players = players.rename(index=str, columns={"Players": "Name"})
players['Name'] = players['Name'].apply(lambda row: unidecode.unidecode(row).replace("-", " "))

In [92]:
players.to_pickle("Players_Historicals_With_Injuries.pkl")